In [1]:
####Import required libraries and start a sagemaker session 
library(reticulate) 
sagemaker <- import('sagemaker')
session <- sagemaker$Session() 
role_arn <- sagemaker$get_execution_role()
bucket = "bdx-demo-sagemaker"
library(readr)
print("Step1")

[1] "Step1"


In [8]:
####Read clinical data from S3, remove rows with NA and generate a new column of obese status based on bmi>30
data_file <- 's3://bdx-demo-sagemaker/clinical_dat.csv'
clidata <- read_csv(file = sagemaker$s3$S3Downloader$read_file(data_file, sagemaker_session=session),
                    col_names = TRUE)
colnames(clidata)
length(clidata$eid)
clidata<-na.omit(clidata)
length(clidata$eid)
clidata$obese <- as.integer(clidata$bmi>30)
colnames(clidata)
print("Step2")

Rows: 442519 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (6): eid, gender, age, bmi, freq, prs

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "eid"    "gender" "age"    "bmi"    "freq"   "prs"

[1] 442519

[1] 429582

[1] "eid"    "gender" "age"    "bmi"    "freq"   "prs"    "obese"

[1] "Step2"


In [9]:
####Do some preliminary analysis of clinical data to ensure revelance of selected features
####Polygenic Risk Scores (prs) was pre-generated from the research project. However, in this demo each
####bmi-associated SNP is treated as an individual predictor
cor.test(clidata$bmi,clidata$age)
cor.test(clidata$bmi,clidata$gender)
cor.test(clidata$bmi,clidata$freq)
cor.test(clidata$bmi,clidata$prs)
summary(lm(bmi~prs+factor(gender)+age+freq,data=clidata))
####Keep only the columns for subsequent steps
sel_cols=c('eid','obese','gender','age','freq')
clidata_1<-clidata[sel_cols]
head(clidata_1)


	Pearson's product-moment correlation

data:  clidata$bmi and clidata$age
t = 31.042, df = 429580, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.04432412 0.05029148
sample estimates:
       cor 
0.04730822 



	Pearson's product-moment correlation

data:  clidata$bmi and clidata$gender
t = 55.679, df = 429580, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.08167694 0.08761484
sample estimates:
       cor 
0.08464664 



	Pearson's product-moment correlation

data:  clidata$bmi and clidata$freq
t = 83.192, df = 429580, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.1229742 0.1288602
sample estimates:
      cor 
0.1259183 



	Pearson's product-moment correlation

data:  clidata$bmi and clidata$prs
t = 42.932, df = 429580, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.06238455 0.06833975
sample estimates:
       cor 
0.06536273 



Call:
lm(formula = bmi ~ prs + factor(gender) + age + freq, data = clidata)

Residuals:
    Min      1Q  Median      3Q     Max 
-16.166  -3.176  -0.687   2.395  48.355 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)     2.190e+01  7.470e-02  293.11   <2e-16 ***
prs             3.690e+00  8.783e-02   42.02   <2e-16 ***
factor(gender)1 1.044e+00  1.457e-02   71.67   <2e-16 ***
age             2.675e-02  8.933e-04   29.95   <2e-16 ***
freq            4.618e-01  4.872e-03   94.79   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.681 on 429577 degrees of freedom
Multiple R-squared:  0.03369,	Adjusted R-squared:  0.03368 
F-statistic:  3744 on 4 and 429577 DF,  p-value: < 2.2e-16


eid,obese,gender,age,freq
<dbl>,<int>,<dbl>,<dbl>,<dbl>
1000015,0,0,64,5
1000027,0,0,60,3
1000039,0,0,58,1
1000040,0,1,66,3
1000053,1,0,67,4
1000064,0,1,64,2


In [10]:
####Read genotype file from S3 and remove rows with NA.
gt_file <- 's3://bdx-demo-sagemaker/obesity.snp.gt'
gtdata<- read_delim(file = sagemaker$s3$S3Downloader$read_file(gt_file, sagemaker_session=session),
                    col_names = TRUE,delim='\t')
gtdata<-na.omit(gtdata)
colnames(gtdata)
####Read a GWAS sumstat file from S3 which is used to code genotypes according to numbers of effect alleles for bmi.
al_file <- 's3://bdx-demo-sagemaker/obesity.sumstat.pos'
aldata<- read_delim(file = sagemaker$s3$S3Downloader$read_file(al_file, sagemaker_session=session),
                    col_names = FALSE,delim='\t')
colnames(aldata)

Rows: 488377 Columns: 23
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (22): rs3101336, rs543874, rs2820292, rs7903146, rs2176598, rs3817334, r...
dbl  (1): ID

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "ID"         "rs3101336"  "rs543874"   "rs2820292"  "rs7903146" 
 [6] "rs2176598"  "rs3817334"  "rs7138803"  "rs11057405" "rs11847697"
[11] "rs9925964"  "rs29941"    "rs2075650"  "rs3810291"  "rs2836754" 
[16] "rs10938397" "rs13107325" "rs11727676" "rs2112347"  "rs1167827" 
[21] "rs2245368"  "rs2033732"  "rs10968576"

Rows: 97 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): X1, X2, X3
dbl (4): X4, X5, X6, X7

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "X1" "X2" "X3" "X4" "X5" "X6" "X7"

In [11]:
####Code the genotypes according to numbers of effect alleles for bmi.
library("stringr")
snp=colnames(gtdata)[-1]
allele=aldata[match(snp,aldata$X1),]$X2
gt_coded=gtdata[,1]
n=length(snp)
for(i in 1:n)
{
  print(i)
  col_ct=str_count(gtdata[[snp[i]]],allele[i])
  gt_coded=cbind(gt_coded,col_ct)
}
colnames(gt_coded)=colnames(gtdata)

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22


In [35]:
####Merge clinical data and gentopye data
colnames(clidata_1)
colnames(gt_coded)
matched_ids=match(clidata_1$eid,gt_coded$ID)
####Examine if there is unmatched IDs
sum(is.na(matched_ids))
####Merge
processed_data=cbind(clidata_1,gt_coded[matched_ids,])
####Double check IDs are matched correctly
head(processed_data)
####Remove duplicate columns
library(dplyr)
undesired <- c('ID')
processed_data <- processed_data %>% select(-one_of(undesired))

[1] "eid"    "obese"  "gender" "age"    "freq"

[1] "ID"         "rs3101336"  "rs543874"   "rs2820292"  "rs7903146" 
 [6] "rs2176598"  "rs3817334"  "rs7138803"  "rs11057405" "rs11847697"
[11] "rs9925964"  "rs29941"    "rs2075650"  "rs3810291"  "rs2836754" 
[16] "rs10938397" "rs13107325" "rs11727676" "rs2112347"  "rs1167827" 
[21] "rs2245368"  "rs2033732"  "rs10968576"

[1] 0

,eid,obese,gender,age,freq,ID,rs3101336,rs543874,rs2820292,rs7903146,⋯,rs3810291,rs2836754,rs10938397,rs13107325,rs11727676,rs2112347,rs1167827,rs2245368,rs2033732,rs10968576
,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
316020,1000015,0,0,64,5,1000015,2,0,2,1,⋯,1,2,1,0,2,2,1,1,2,0
430032,1000027,0,0,60,3,1000027,2,1,0,1,⋯,2,1,1,0,2,1,2,0,0,0
250516,1000039,0,0,58,1,1000039,2,0,0,1,⋯,2,0,0,0,2,1,2,1,1,1
478428,1000040,0,1,66,3,1000040,1,0,2,1,⋯,1,1,1,0,2,1,2,0,2,0
249258,1000053,1,0,67,4,1000053,1,1,1,1,⋯,2,1,2,1,2,2,1,0,2,0
195331,1000064,0,1,64,2,1000064,1,0,0,2,⋯,1,2,2,1,1,0,2,1,2,1



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [37]:
####Save the dataset for training and upload to S3
write_csv(processed_data, 'obesity_training_data.csv', col_names = FALSE)
s3_train <- session$upload_data(path = 'obesity_training_data.csv', 
                                bucket = bucket,
                                key_prefix = 'data')